## Preparación de los datos

### Importar librerías

In [1]:
import pandas as pd
import dask.dataframe as dd

### Cargar datos

In [2]:
clients= pd.read_csv('/Users/samanthaestudillo/Desktop/BimboETL/cliente_tabla.csv')
products= pd.read_csv('/Users/samanthaestudillo/Desktop/BimboETL/producto_tabla.csv')
sample_submission= pd.read_csv('/Users/samanthaestudillo/Desktop/BimboETL/sample_submission.csv') 
test= pd.read_csv('/Users/samanthaestudillo/Desktop/BimboETL/test.csv')
town_state= pd.read_csv('/Users/samanthaestudillo/Desktop/BimboETL/town_state.csv') 

train= dd.read_csv('/Users/samanthaestudillo/Desktop/BimboETL/train.csv')

### Optimizar datos

#### Tabla de clientes

In [3]:
clients.info(memory_usage='deep')
clients

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935362 entries, 0 to 935361
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Cliente_ID     935362 non-null  int64 
 1   NombreCliente  935362 non-null  object
dtypes: int64(1), object(1)
memory usage: 70.8 MB


,Cliente_ID,NombreCliente
0,0,SIN NOMBRE
1,1,OXXO XINANTECATL
2,2,SIN NOMBRE
3,3,EL MORENO
4,4,SDN SER DE ALIM CUERPO SA CIA DE INT
...,...,...
935357,11011586,OXXO PETEN
935358,11693264,SUPER ABARROTES MARY
935359,19988629,NO IDENTIFICADO
935360,99999999,NO IDENTIFICADO


In [4]:
# renombrar columnas 
clients= clients.rename(columns={'Cliente_ID': 'client_id', 'NombreCliente': 'client_name'})

# estandarizar nombres de clientes
clients['client_name']= clients['client_name'].str.strip()  
clients['client_name']= clients['client_name'].str.upper()

# verificar valores nulos
print(f"Valores nulos \n{clients.isnull().sum()} \n")

# verificar valores duplicados en client_id
duplicates_clients_id= clients.duplicated(subset='client_id').sum()
print(f"Duplicados encontrados y eliminados: {duplicates_clients_id}")

# eliminar valores duplicados
clients = clients.drop_duplicates(subset='client_id').reset_index(drop = True)

Valores nulos 
client_id      0
client_name    0
dtype: int64 

Duplicados encontrados y eliminados: 4862


In [5]:
no_names = ['SIN NOMBRE', 'NO IDENTIFICADO']

# eliminar filas 
clients_clean = clients[~clients['client_name'].isin(no_names)].reset_index(drop = True)

# verificar cuántas filas se eliminaron
removed_rows = clients.shape[0] - clients_clean.shape[0]
print(f"Se eliminaron {removed_rows} filas que contenian SIN NOMBRE o NO IDENTIFICADO. \n")

Se eliminaron 282026 filas que contenian SIN NOMBRE o NO IDENTIFICADO. 



1. **Renombrado de columnas**: Las columnas fueron renombradas para una mejor comprensión: `Cliente_ID` a `client_id` y `NombreCliente` a `client_name`.

2. **Estandarización de nombres**: Los nombres de los clientes fueron estandarizados eliminando espacios en blanco y convirtiéndolos a mayúsculas.

3. **Valores nulos**: Se verificaron los valores nulos asegurando que no haya datos faltantes en las columnas.

4. **Duplicados**: Se identificaron y eliminaron valores duplicados basados en `client_id` para asegurar la unicidad de los registros de clientes.

5. **Filas sin nombre**: Se eliminaron filas con nombres como 'SIN NOMBRE' o 'NO IDENTIFICADO' para mejorar la calidad de los datos.

#### Tabla de productos

In [6]:
products.info(memory_usage='deep')
products

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2592 entries, 0 to 2591
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Producto_ID     2592 non-null   int64 
 1   NombreProducto  2592 non-null   object
dtypes: int64(1), object(1)
memory usage: 254.3 KB


,Producto_ID,NombreProducto
0,0,NO IDENTIFICADO 0
1,9,Capuccino Moka 750g NES 9
2,41,Bimbollos Ext sAjonjoli 6p 480g BIM 41
3,53,Burritos Sincro 170g CU LON 53
4,72,Div Tira Mini Doradita 4p 45g TR 72
...,...,...
2587,49992,Tostado Integral 180g MTA WON 49992
2588,49993,Tostado Integral 180g TAB WON 49993
2589,49994,Tostado Int 0pct Grasa Azuc 200g WON 49994
2590,49996,Tostado Int 0pct Grasa Azuc 200g MTA WON 49996


In [7]:
# renombrar columnas 
products = products.rename(columns={'Producto_ID': 'product_id', 'NombreProducto': 'product_name'})

# estandarizar nombres de productos
products['product_name'] = products['product_name'].str.strip()  
products['product_name'] = products['product_name'].str.upper()

# verificar valores nulos
print(f"Valores nulos \n{products.isnull().sum()}, \n")
    
# verificar valores duplicados en product_id
duplicates_product_id= products.duplicated(subset='product_id').sum()
print(f"Duplicados encontrados y eliminados: {duplicates_product_id}")

# eliminar valores duplicados
products = products.drop_duplicates(subset='product_id').reset_index(drop = True)

# eliminar product_id del final de product_name
products['product_name'] = products['product_name'].apply(lambda x: x.rsplit(' ', 1)[0].strip())

Valores nulos 
product_id      0
product_name    0
dtype: int64, 

Duplicados encontrados y eliminados: 0


In [8]:
# eliminar filas 
products_clean = products[~products['product_name'].isin(no_names)].reset_index(drop = True)

# verificar cuántas filas se eliminaron
removed_rows_products = products.shape[0] - products_clean.shape[0]
print(f"Se eliminaron {removed_rows_products} filas que contenian SIN NOMBRE o NO IDENTIFICADO. \n")

Se eliminaron 1 filas que contenian SIN NOMBRE o NO IDENTIFICADO. 



1. **Renombrado de columnas**: Las columnas fueron renombradas para una mejor comprensión: `Producto_ID` a `product_id` y `NombreProducto` a `product_name`.

2. **Estandarización de nombres**: Los nombres de los productos fueron estandarizados eliminando espacios en blanco y convirtiéndolos a mayúsculas.

3. **Valores nulos**: Se verificaron los valores nulos en el DataFrame, asegurando que no haya datos faltantes.

4. **Duplicados**: Se identificaron valores duplicados basados en `product_id` y se reportó que no hubo duplicados encontrados.

5. **Eliminación de sufijo en nombres de productos**: Se eliminaron los sufijos `product_id` del final de `product_name` para mantener los nombres de productos más limpios y claros.

6. **Filas sin nombre**: Se eliminaron filas con nombres como 'SIN NOMBRE' o 'NO IDENTIFICADO' para mejorar la calidad de los datos.

#### Tabla de muestras

In [9]:
sample_submission.info(memory_usage='deep')
sample_submission

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999251 entries, 0 to 6999250
Data columns (total 2 columns):
 #   Column             Dtype
---  ------             -----
 0   id                 int64
 1   Demanda_uni_equil  int64
dtypes: int64(2)
memory usage: 106.8 MB


,id,Demanda_uni_equil
0,0,7
1,1,7
2,2,7
3,3,7
4,4,7
...,...,...
6999246,6999246,7
6999247,6999247,7
6999248,6999248,7
6999249,6999249,7


In [10]:
# renombrar columnas 
sample_submission = sample_submission.rename(columns={'Demanda_uni_equil': 'adjusted_demand_units'})

1. **Renombrado de columnas**: Solo la columna `Demanda_uni_equil` fue renombrada a `adjusted_demand_units` para una mejor comprensión y sintonía con el resto de los DataFrame.

#### Tabla de ventas

In [11]:
test.info(memory_usage='deep')
test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999251 entries, 0 to 6999250
Data columns (total 7 columns):
 #   Column       Dtype
---  ------       -----
 0   id           int64
 1   Semana       int64
 2   Agencia_ID   int64
 3   Canal_ID     int64
 4   Ruta_SAK     int64
 5   Cliente_ID   int64
 6   Producto_ID  int64
dtypes: int64(7)
memory usage: 373.8 MB


,id,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277
...,...,...,...,...,...,...,...
6999246,6999246,11,2057,1,1153,4379638,1232
6999247,6999247,10,1334,1,2008,970421,43069
6999248,6999248,11,1622,1,2869,192749,30532
6999249,6999249,11,1636,1,4401,286071,35107


In [12]:
# renombrar columnas 
test = test.rename(columns={
    'Semana': 'week',
    'Agencia_ID': 'agency_id',
    'Canal_ID': 'channel_id',
    'Ruta_SAK': 'sak_route',
    'Cliente_ID': 'client_id',
    'Producto_ID': 'product_id'
})

# verificar valores nulos
print(f" Valores nulos \n{test.isnull().sum()}\n")

# verificar valores duplicados en id
duplicates_test_id= test.duplicated(subset='id').sum()
print(f"Duplicados encontrados y eliminados: {duplicates_test_id}")

 Valores nulos 
id            0
week          0
agency_id     0
channel_id    0
sak_route     0
client_id     0
product_id    0
dtype: int64

Duplicados encontrados y eliminados: 0


1. **Renombrado de columnas**: Las columnas fueron renombradas para una mejor comprensión, asi como mantener el formato que llevan las columnas en los otros DFs.

3. **Valores nulos**: Se verificaron los valores nulos en el DataFrame, asegurando que no haya datos faltantes.

4. **Duplicados**: Se identificaron valores duplicados basados en `id` y se reportó que no hubo duplicados encontrados.

#### Tabla de localidades

In [13]:
town_state.info(memory_usage='deep')
town_state

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 790 entries, 0 to 789
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Agencia_ID  790 non-null    int64 
 1   Town        790 non-null    object
 2   State       790 non-null    object
dtypes: int64(1), object(2)
memory usage: 121.6 KB


,Agencia_ID,Town,State
0,1110,2008 AG. LAGO FILT,"MÉXICO, D.F."
1,1111,2002 AG. AZCAPOTZALCO,"MÉXICO, D.F."
2,1112,2004 AG. CUAUTITLAN,ESTADO DE MÉXICO
3,1113,2008 AG. LAGO FILT,"MÉXICO, D.F."
4,1114,2029 AG.IZTAPALAPA 2,"MÉXICO, D.F."
...,...,...,...
785,25729,2572 TIJUANA EL FLORIDO,BAJA CALIFORNIA NORTE
786,25739,2573 TIJUANA GATO BRONCO,BAJA CALIFORNIA NORTE
787,25749,2574 TIJUANA PACIFICO,BAJA CALIFORNIA NORTE
788,25759,2575 TIJUANA ROSARITO,BAJA CALIFORNIA NORTE


In [14]:
# renombrar columnas
town_state = town_state.rename(columns={
    'Agencia_ID': 'agency_id',
    'Town': 'town',
    'State': 'state'
})

# estandarizar ciudades y estados
town_state['town'] = town_state['town'].str.strip()  
town_state['town'] = town_state['town'].str.upper()
town_state['state'] = town_state['state'].str.strip()  
town_state['state'] = town_state['state'].str.upper()

# verificar valores nulos
print(f" Valores nulos \n{town_state.isnull().sum()} \n")

# verificar valores duplicados en agency_id
duplicates_agency_id= town_state.duplicated(subset='agency_id').sum()
print(f"Duplicados encontrados y eliminados: {duplicates_agency_id} \n")

# extraer los primeros 4 dígitos de la columna 'Town'
town_state['cp'] = town_state['town'].str.slice(stop=4)
town_state['town'] = town_state['town'].str.slice(start=4)

town_state.head()

 Valores nulos 
agency_id    0
town         0
state        0
dtype: int64 

Duplicados encontrados y eliminados: 0 



,agency_id,town,state,cp
0,1110,AG. LAGO FILT,"MÉXICO, D.F.",2008
1,1111,AG. AZCAPOTZALCO,"MÉXICO, D.F.",2002
2,1112,AG. CUAUTITLAN,ESTADO DE MÉXICO,2004
3,1113,AG. LAGO FILT,"MÉXICO, D.F.",2008
4,1114,AG.IZTAPALAPA 2,"MÉXICO, D.F.",2029


1. **Renombrado de columnas**: Las columnas fueron renombradas para una mejor comprensión: `Agencia_ID` a `agency_id`, `Town` a `town` y `State` a `state`.

2. **Estandarización de nombres**: Los nombres de las ciudades y estados fueron estandarizados eliminando espacios en blanco y convirtiéndolos a mayúsculas.

3. **Valores nulos**: Se verificaron los valores nulos asegurando que no haya datos faltantes.

4. **Duplicados**: Se identificaron valores duplicados basados en `agency_id` y se reportó que no hubo duplicados encontrados.

5. **Prefijo en las ciudades**: Se movieron los 4 primeros numeros antes del nombre en la columna `town` a una nueva columna `cp` que hace referencia al codigo postal. 

#### Tabla ventas/ devoluciones/ demanda

In [15]:
train.dtypes

Semana                 int64
Agencia_ID             int64
Canal_ID               int64
Ruta_SAK               int64
Cliente_ID             int64
Producto_ID            int64
Venta_uni_hoy          int64
Venta_hoy            float64
Dev_uni_proxima        int64
Dev_proxima          float64
Demanda_uni_equil      int64
dtype: object

In [16]:
# Convertir tipos de datos
train['Semana'] = train['Semana'].astype('int32')
train['Agencia_ID'] = train['Agencia_ID'].astype('int32')
train['Canal_ID'] = train['Canal_ID'].astype('int32')
train['Ruta_SAK'] = train['Ruta_SAK'].astype('int32')
train['Cliente_ID'] = train['Cliente_ID'].astype('int32')
train['Producto_ID'] = train['Producto_ID'].astype('int32')
train['Venta_uni_hoy'] = train['Venta_uni_hoy'].astype('int32')
train['Dev_uni_proxima'] = train['Dev_uni_proxima'].astype('int32')
train['Demanda_uni_equil'] = train['Demanda_uni_equil'].astype('int32')

train['Venta_hoy'] = train['Venta_hoy'].astype('float32')
train['Dev_proxima'] = train['Dev_proxima'].astype('float32')

In [17]:
train.head()

,Semana,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID,Producto_ID,Venta_uni_hoy,Venta_hoy,Dev_uni_proxima,Dev_proxima,Demanda_uni_equil
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.320000,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.520000,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.920000,0,0.0,3


In [18]:
# Muestra el número de filas y columnas
num_rows = train.shape[0].compute()
num_cols = train.shape[1]

print(f"Shape: ({num_rows}, {num_cols})")

Shape: (74180464, 11)


In [19]:
# renombrar columnas 
train = train.rename(columns={
    'Semana': 'week',
    'Agencia_ID': 'agency_id',
    'Canal_ID': 'channel_id',
    'Ruta_SAK': 'sak_route',
    'Cliente_ID': 'client_id',
    'Producto_ID': 'product_id',
    'Venta_uni_hoy': 'units_sold_today',
    'Venta_hoy': 'sales_today',
    'Dev_uni_proxima': 'units_returned_next_period',
    'Dev_proxima': 'returns_next_period',
    'Demanda_uni_equil': 'adjusted_demand_units'
})

# verificar valores nulos
print(train.isnull().sum().compute())

week                          0
agency_id                     0
channel_id                    0
sak_route                     0
client_id                     0
product_id                    0
units_sold_today              0
sales_today                   0
units_returned_next_period    0
returns_next_period           0
adjusted_demand_units         0
dtype: int64


In [20]:
train.describe().compute()

,week,agency_id,channel_id,sak_route,client_id,product_id,units_sold_today,sales_today,units_returned_next_period,returns_next_period,adjusted_demand_units
count,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07,7.418046e+07
mean,5.950021e+00,2.536509e+03,1.383181e+00,2.114855e+03,1.802119e+06,2.084081e+04,7.310163e+00,6.854452e+01,1.302577e-01,1.243248e+00,7.224564e+00
std,2.013175e+00,4.075124e+03,1.463266e+00,1.487744e+03,2.349577e+06,1.866392e+04,2.196734e+01,3.389795e+02,2.932320e+01,3.921552e+01,2.177119e+01
min,3.000000e+00,1.110000e+03,1.000000e+00,1.000000e+00,2.600000e+01,4.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.000000e+00,1.332000e+03,1.000000e+00,1.211000e+03,4.827250e+05,2.425000e+03,2.000000e+00,1.886000e+01,0.000000e+00,0.000000e+00,2.000000e+00
50%,6.000000e+00,1.625000e+03,1.000000e+00,1.602000e+03,1.448102e+06,3.530500e+04,4.000000e+00,3.430000e+01,0.000000e+00,0.000000e+00,4.000000e+00
75%,8.000000e+00,2.054000e+03,1.000000e+00,2.905000e+03,2.491048e+06,4.320100e+04,8.000000e+00,6.417000e+01,0.000000e+00,0.000000e+00,8.000000e+00
max,9.000000e+00,2.575900e+04,1.100000e+01,9.991000e+03,2.015152e+09,4.999700e+04,7.200000e+03,6.473600e+05,2.500000e+05,1.307600e+05,5.000000e+03


1. **Convertir tipos de datos**: Se convirtieron varias columnas a tipos de datos más eficientes y apropiados:
     - Columnas numéricas se convirtieron a `int32` para ocupar menos memoria.
     - Columnas de ventas y devoluciones se convirtieron a `float32`.

2. **Mostrar información**: Se muestra el número total de filas y columnas.

3. **Renombrar columnas**: Se renombraron las columnas para mejorar la claridad y consistencia de los nombres.

4. **Valores nulos**: Se revisó la cantidad de valores nulos por columna para identificar cualquier dato faltante que necesite ser tratado.

5. **Descripción estadística del DataFrame**: Se calculó una descripción estadística, proporcionando medidas resumidas como media, desviación estándar, y cuartiles para columnas numéricas.


### DataFrame final

#### clients_clean: 
Este archivo contiene información sobre los clientes de Grupo Bimbo. Las columnas son:
1. client_id: Identificador único para cada cliente.
2. client_name: Nombre del cliente.

In [21]:
clients_clean

,client_id,client_name
0,1,OXXO XINANTECATL
1,3,EL MORENO
2,4,SDN SER DE ALIM CUERPO SA CIA DE INT
3,5,LA VAQUITA
4,6,LUPITA
...,...,...
648469,10351802,OXXO IDOLO II
648470,10351808,OXXO TUXPAM TAMIAHUA
648471,10351814,OXXO DE LA LLAVE
648472,11011586,OXXO PETEN


#### products_clean: 
Este archivo contiene información sobre los productos de Grupo Bimbo. Las columnas son:
1. product_id: Identificador único para cada cliente.
2. product_name: Nombre del cliente.

In [22]:
products_clean

,product_id,product_name
0,9,CAPUCCINO MOKA 750G NES
1,41,BIMBOLLOS EXT SAJONJOLI 6P 480G BIM
2,53,BURRITOS SINCRO 170G CU LON
3,72,DIV TIRA MINI DORADITA 4P 45G TR
4,73,PAN MULTIGRANO LINAZA 540G BIM
...,...,...
2586,49992,TOSTADO INTEGRAL 180G MTA WON
2587,49993,TOSTADO INTEGRAL 180G TAB WON
2588,49994,TOSTADO INT 0PCT GRASA AZUC 200G WON
2589,49996,TOSTADO INT 0PCT GRASA AZUC 200G MTA WON


#### test: 
Este archivo contiene información relacionada con las ventas de Grupo Bimbo. Las columnas son:
1. id: Identificador único para cada registro.
2. week: Número de la semana en la que se realizaron las ventas.
3. agency_id: Identificador de la agencia que realizó la venta.
4. channel_id: Identificador del canal de ventas.
5. sak_route: Identificador de la ruta de distribución.
6. client_id: Identificador del cliente al que se le vendió el producto.
7. product_id: Identificador del producto vendido.

In [23]:
test

,id,week,agency_id,channel_id,sak_route,client_id,product_id
0,0,11,4037,1,2209,4639078,35305
1,1,11,2237,1,1226,4705135,1238
2,2,10,2045,1,2831,4549769,32940
3,3,11,1227,1,4448,4717855,43066
4,4,11,1219,1,1130,966351,1277
...,...,...,...,...,...,...,...
6999246,6999246,11,2057,1,1153,4379638,1232
6999247,6999247,10,1334,1,2008,970421,43069
6999248,6999248,11,1622,1,2869,192749,30532
6999249,6999249,11,1636,1,4401,286071,35107


#### town_state:
Este archivo contiene información sobre las agencias de Grupo Bimbo en diferentes localidades. Las columnas son:
1. agency_id: Identificador único para cada agencia.
2. town: Nombre de la ciudad donde se encuentra la agencia.
3. state: Estado donde se encuentra la ciudad de la agencia.
4. cp: Codigo postal donde se encuentra la ciudad de la agencia.

In [24]:
town_state

,agency_id,town,state,cp
0,1110,AG. LAGO FILT,"MÉXICO, D.F.",2008
1,1111,AG. AZCAPOTZALCO,"MÉXICO, D.F.",2002
2,1112,AG. CUAUTITLAN,ESTADO DE MÉXICO,2004
3,1113,AG. LAGO FILT,"MÉXICO, D.F.",2008
4,1114,AG.IZTAPALAPA 2,"MÉXICO, D.F.",2029
...,...,...,...,...
785,25729,TIJUANA EL FLORIDO,BAJA CALIFORNIA NORTE,2572
786,25739,TIJUANA GATO BRONCO,BAJA CALIFORNIA NORTE,2573
787,25749,TIJUANA PACIFICO,BAJA CALIFORNIA NORTE,2574
788,25759,TIJUANA ROSARITO,BAJA CALIFORNIA NORTE,2575


#### train: 
Este archivo contiene información detallada sobre las ventas, devoluciones y demanda de los productos de Grupo Bimbo. Las columnas son:
1. week: Número de la semana en la que se realizaron las ventas.
2. agency_id: Identificador de la agencia que realizó la venta.
3. channel_id: Identificador del canal de ventas.
4. sak_route: Identificador de la ruta de distribución.
5. client_id: Identificador del cliente al que se le vendió el producto.
6. product_id: Identificador del producto vendido.
7. units_sold_today: Unidades vendidas del producto en el día.
8. sales_today: Monto total de las ventas del producto en el día (en la moneda local).
9. units_returned_next_period: Unidades del producto devueltas en el periodo siguiente.
10. returns_next_period: Monto total de las devoluciones del producto en el periodo siguiente (en la moneda local).
11.  adjusted_demand_units: Demanda ajustada del producto (unidades), calculada a partir de las ventas y devoluciones.

In [25]:
train.head()

,week,agency_id,channel_id,sak_route,client_id,product_id,units_sold_today,sales_today,units_returned_next_period,returns_next_period,adjusted_demand_units
0,3,1110,7,3301,15766,1212,3,25.139999,0,0.0,3
1,3,1110,7,3301,15766,1216,4,33.520000,0,0.0,4
2,3,1110,7,3301,15766,1238,4,39.320000,0,0.0,4
3,3,1110,7,3301,15766,1240,4,33.520000,0,0.0,4
4,3,1110,7,3301,15766,1242,3,22.920000,0,0.0,3


#### sample_submission:
Este archivo contiene información sobre la demanda ajustada del producto.Las columnas son:
1. id: Identificador único para cada registro.
2. adjusted_demand_units: Demanda ajustada del producto (unidades), calculada a partir de las ventas y devoluciones.

In [26]:
sample_submission

,id,adjusted_demand_units
0,0,7
1,1,7
2,2,7
3,3,7
4,4,7
...,...,...
6999246,6999246,7
6999247,6999247,7
6999248,6999248,7
6999249,6999249,7


### Convertir a CSV

In [27]:
clients_clean.to_csv('clients.csv', index=False)
products_clean.to_csv('products.csv', index=False)
test.to_csv('test.csv', index=False)
town_state.to_csv('town_state.csv', index=False)
sample_submission.to_csv('sample_submission.csv', index=False)